In [1]:
import pandas as pd

In [50]:
df = pd.read_csv("C:/Users/Jorge Schimpf/Documents/Henry/proyecto integrador 1/Movies/movies_dataset.csv")
df.shape

C:\Users\Jorge Schimpf\AppData\Local\Temp\ipykernel_1760\3594049021.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Jorge Schimpf/Documents/Henry/proyecto integrador 1/Movies/movies_dataset.csv")


(45466, 24)

In [51]:
df2 = df[['id', 'title', 'release_date', 'belongs_to_collection', 'budget', 'revenue', 'genres', 'original_language', 'overview',
        'popularity', 'production_companies',
        'production_countries', 'runtime',
        'spoken_languages', 'status', 'tagline',
        'vote_average', 'vote_count']]
df2.head(1) 

,id,title,release_date,belongs_to_collection,budget,revenue,genres,original_language,overview,popularity,production_companies,production_countries,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,862,Toy Story,1995-10-30,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,373554033.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,7.7,5415.0


In [57]:
df2["release_date"] = pd.to_datetime(df2["release_date"], 'coerce', format='mixed')
df_cleandates = df2.dropna(axis=0, subset=['release_date'])
df_cleandates.to_csv("movies_dataset_dates_cleared.csv", index=False)
df_cleandates.shape

(45376, 18)

In [2]:
df = pd.read_csv("../Movies/movies_dataset_dates_cleared.csv", index_col="id")

In [3]:
df["revenue"] = df["revenue"].fillna(0)
df["budget"] = df["budget"].fillna(0)
print(df["budget"].count(), df["revenue"].count())

45376 45376


In [4]:
#for a return column
def get_return(df_row):
    ret = 0
    if df_row['budget'] != 0:
        ret = float(df_row['revenue'])/float(df_row['budget'])
    return ret

In [5]:
df["return"] = df.apply(get_return, axis=1)
df.to_csv("../Movies/movies_plus_return.csv")
df.head(1)

,title,release_date,belongs_to_collection,budget,revenue,genres,original_language,overview,popularity,production_companies,production_countries,runtime,spoken_languages,status,tagline,vote_average,vote_count,return
id,,,,,,,,,,,,,,,,,,
862,Toy Story,1995-10-30,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,373554033.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,7.7,5415.0,12.451801


In [6]:
df_remake = df[["belongs_to_collection", "production_companies", "production_countries", "spoken_languages", "genres"]]
df_remake.shape

(45376, 5)

In [7]:
df_remake.to_csv("prod.csv")

## df_remake como base para extraer indices

In [8]:
import numpy as np
collections_ids = []

dict_default = {'id': [], 'name': [], 'poster_path': [], 'backdrop_path': []}
df_collections = []


def get_collection(row)-> None | np.int64:
    if not pd.notna(row['belongs_to_collection']) or row['belongs_to_collection'] =="" or type(row['belongs_to_collection']) != str:
        return np.nan
    
    dicti = eval(row['belongs_to_collection'])
    if dicti is None:
        return np.nan
    global df_collections
    if dicti['id'] not in collections_ids:
        collections_ids.append(dicti['id'])
        df_collections.append(dicti)
        
    return int(dicti['id'])


print(df_collections)

[]


In [9]:
df_remake['belongs_to_collection'] = df_remake.apply(get_collection, axis=1).astype('Int64')
print(df_remake)
print(df_collections)
print(collections_ids)

        belongs_to_collection  \
id                              
862                     10194   
8844                     <NA>   
15602                  119050   
31357                    <NA>   
11862                   96871   
...                       ...   
30840                    <NA>   
111109                   <NA>   
67758                    <NA>   
227506                   <NA>   
461257                   <NA>   

                                     production_companies  \
id                                                          
862        [{'name': 'Pixar Animation Studios', 'id': 3}]   
8844    [{'name': 'TriStar Pictures', 'id': 559}, {'na...   
15602   [{'name': 'Warner Bros.', 'id': 6194}, {'name'...   
31357   [{'name': 'Twentieth Century Fox Film Corporat...   
11862   [{'name': 'Sandollar Productions', 'id': 5842}...   
...                                                   ...   
30840   [{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...   
111109             [{

C:\Users\Jorge Schimpf\AppData\Local\Temp\ipykernel_8232\2387820110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_remake['belongs_to_collection'] = df_remake.apply(get_collection, axis=1).astype('Int64')


In [10]:
df_remake.to_csv("prod2.csv")

In [11]:
df_collec = pd.DataFrame(df_collections)
df_collec = df_collec.set_index("id")
df_collec

,name,poster_path,backdrop_path
id,,,
10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
645,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
117693,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
...,...,...,...
104774,Tomtar och Trolltyg Collection,/5UMq1DmJva3l7wg3VK641Sfn9D.jpg,None
400500,Чебурашка и крокодил Гена,/epKHY3ZHJkTF7U3YAXCrjtmnC99.jpg,/zBStJBeBsnRMTyCLBSWlhLWB4XW.jpg
148603,Ducobu Collection,/rd7AWZUy2QFPIblNWToVmdfXQcA.jpg,/7mzKmoIrvGapvsSbAVlX4HtCnFj.jpg


In [13]:
df_collec.to_csv("collections.csv")

## For production_companies into companies.csv

In [29]:
df_remake = pd.read_csv("prod2.csv", index_col=["id"])
df_remake.head(1)

,belongs_to_collection,production_companies,production_countries,spoken_languages,genres
id,,,,,
862,10194.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
8844,NaN,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
15602,119050.0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
31357,NaN,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
11862,96871.0,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...
30840,NaN,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
111109,NaN,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]","[{'iso_639_1': 'tl', 'name': ''}]","[{'id': 18, 'name': 'Drama'}]"
67758,NaN,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."


In [ ]:
import numpy as np
ids = []
df_companies = []


def get_companies(row)-> None | str:
    if not pd.notna(row['production_companies']) or row['production_companies'] =="" or type(row['production_companies']) != str:
        return None
    
    listi = eval(row['production_companies'])
    
    if listi is None:
        return None
    
    
    global df_companies
    result = []
    for i in listi:
        result.append(int(i['id']))
        if i['id'] not in ids:
            ids.append(i['id'])
            df_companies.append(i)
        
    return str(result)


print(df_companies)

[]


In [ ]:
df_remake['production_companies'] = df_remake.apply(get_companies, axis=1).astype(str)
print(df_remake)
print(df_companies)
print(ids)

        belongs_to_collection         production_companies  \
id                                                           
862                   10194.0                          [3]   
8844                      NaN           [559, 2550, 10201]   
15602                119050.0                [6194, 19464]   
31357                     NaN                        [306]   
11862                 96871.0                 [5842, 9195]   
...                       ...                          ...   
30840                     NaN  [7025, 10163, 16323, 38978]   
111109                    NaN                      [19653]   
67758                     NaN                       [6165]   
227506                    NaN                      [88753]   
461257                    NaN                           []   

                                     production_countries  \
id                                                          
862     [{'iso_3166_1': 'US', 'name': 'United States o...   
8844    [{

In [32]:
df_remake.to_csv("prod3.csv")

In [33]:
df_collec = pd.DataFrame(df_companies)
df_collec = df_collec.set_index("id")
df_collec

,name
id,
3,Pixar Animation Studios
559,TriStar Pictures
2550,Teitler Film
10201,Interscope Communications
6194,Warner Bros.
...,...
85400,He and She Films
27570,Neptune Salad Entertainment
27571,Pirie Productions


In [34]:
df_collec.to_csv("companies.csv")

In [ ]:
import pandas as pd
import numpy as np

pf = "prod3.csv"
id_name = 'iso_3166_1'
nf = "prod4.csv"

col = 'production_countries'

df_remake = pd.read_csv(pf, index_col=["id"])

ids = []
df_companies = []


def get_companies(row)-> None | str:
    if not pd.notna(row[col]) or row[col] =="" or type(row[col]) != str:
        return None
    
    listi = eval(row[col])
    
    if listi is None:
        return None
    
    
    global df_companies
    result = []
    for i in listi:
        result.append(i[id_name])
        if i[id_name] not in ids:
            ids.append(i[id_name])
            df_companies.append(i)
        
    return str(result)

df_remake[col] = df_remake.apply(get_companies, axis=1)

df_remake.to_csv(nf)


df_collec = pd.DataFrame(df_companies)
df_collec = df_collec.set_index(id_name)

df_collec.to_csv("countries.csv")

In [ ]:
pf = "prod4.csv"
id_name = 'iso_639_1'
nf = "prod5.csv"
finale = "languages.csv"

col = 'spoken_languages'

df_remake = pd.read_csv(pf, index_col=["id"])

ids = []
df_companies = []


def get_companies(row)-> None | str:
    if not pd.notna(row[col]) or row[col] =="" or type(row[col]) != str:
        return None
    
    listi = eval(row[col])
    
    if listi is None:
        return None
    
    
    global df_companies
    result = []
    for i in listi:
        result.append(i[id_name])
        if i[id_name] not in ids:
            ids.append(i[id_name])
            df_companies.append(i)
        
    return str(result)

df_remake[col] = df_remake.apply(get_companies, axis=1)

df_remake.to_csv(nf)


df_collec = pd.DataFrame(df_companies)
df_collec = df_collec.set_index(id_name)

df_collec.to_csv(finale)

In [ ]:
pf = "prod5.csv"
id_name = 'id'
nf = "prod6.csv"
finale = "genres.csv"

col = 'genres'

df_remake = pd.read_csv(pf, index_col=["id"])

ids = []
df_companies = []


def get_companies(row)-> None | str:
    if not pd.notna(row[col]) or row[col] =="" or type(row[col]) != str:
        return None
    
    listi = eval(row[col])
    
    if listi is None:
        return None
    
    
    global df_companies
    result = []
    for i in listi:
        result.append(int(i[id_name]))
        if i[id_name] not in ids:
            ids.append(i[id_name])
            df_companies.append(i)
        
    return str(result)

df_remake[col] = df_remake.apply(get_companies, axis=1)

df_remake.to_csv(nf)


df_collec = pd.DataFrame(df_companies)
df_collec = df_collec.set_index(id_name)

df_collec.to_csv(finale)

In [3]:
df = pd.read_csv("../Movies/movies_plus_return.csv", index_col=["id"])
df.head(1)

,title,release_date,belongs_to_collection,budget,revenue,genres,original_language,overview,popularity,production_companies,production_countries,runtime,spoken_languages,status,tagline,vote_average,vote_count,return
id,,,,,,,,,,,,,,,,,,
862,Toy Story,1995-10-30,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,373554033.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,7.7,5415.0,12.451801


In [2]:
finale_df = pd.read_csv("prod6.csv", index_col=["id"])
finale_df["belongs_to_collection"] = finale_df["belongs_to_collection"].astype("Int64")

In [4]:
df[["belongs_to_collection", "production_companies", "production_countries", "spoken_languages", "genres"]] = finale_df[["belongs_to_collection", "production_companies", "production_countries", "spoken_languages", "genres"]]

In [5]:
df.to_csv("movies.csv",)

In [2]:
import pandas as pd
import numpy as np
import ast


In [6]:
pf = "credits.csv"
creditos = pd.read_csv(pf).drop(columns="crew")

In [28]:
crewmates = pd.read_csv(pf).drop(columns="cast")

In [7]:
movies = pd.read_csv("datasets/movies_final.csv", index_col= ["id"]).drop(columns=['release_date','belongs_to_collection','budget','revenue','genres','original_language','overview','popularity','production_companies',"production_countries","runtime","spoken_languages","status","tagline",])

In [ ]:
finale = "cast.csv"

fields = ["gender", "name"]
df_companies = {}
performances = {}

def get_companies(row):
    if not pd.notna(row["cast"]) or type(row["cast"]) != str:
        return
    
    listi = ast.literal_eval(row["cast"])
    
    if listi is None:
        return
    
    global performances
    
    for i in listi:
        defdic = {}
        if i["id"] in list(performances.keys()):
            performances[i["id"]] += [row["id"]]
        else:
            for k in fields:
                if k in i:
                    defdic[k] = i[k]
            performances[i["id"]] = [row["id"],]
            df_companies[i["id"]] = defdic



In [ ]:
creditos.apply(get_companies, axis=1)


In [ ]:
performances

In [ ]:
df_collec = pd.DataFrame.from_dict(df_companies,orient="index")
df_collec.set_index("id", inplace=True).head(1)

In [23]:
for k in performances.keys():
    performances[k] = str(performances[k])

In [ ]:
df_collec["performances"] = pd.DataFrame.from_dict(performances, orient="index", dtype=str)

In [27]:

df_collec = df_collec.set_index("id")

df_collec.to_csv(finale)

In [13]:
df_collec2 = df_collec

In [18]:
df_collec2.to_csv("cast.csv")

In [20]:
movies = pd.read_csv("datasets/movies.csv")
movies["belongs_to_collection"] = movies["belongs_to_collection"].astype("Int64")
movies.head(1)

,id,title,release_date,belongs_to_collection,budget,revenue,genres,original_language,overview,popularity,production_companies,production_countries,runtime,spoken_languages,status,tagline,vote_average,vote_count,return
0,862,Toy Story,1995-10-30,10194,30000000,373554033.0,"[16, 35, 10751]",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[3],['US'],81.0,['en'],Released,NaN,7.7,5415.0,12.451801


In [22]:
movies["release_year"] = pd.to_datetime(movies["release_date"]).dt.year

In [30]:
#movies.set_index("id", inplace = True)
movies.to_csv("datasets/movies.csv")

In [2]:
movies = pd.read_csv("datasets/movies.csv", index_col=["id"])

In [3]:
pop_mean = movies["popularity"].mean()
pop_mean

np.float64(2.926457567502645)

In [4]:
filtered_movies = movies[movies["popularity"]>pop_mean]
filtered_movies.shape

(13051, 19)

In [5]:
filtered_movies.to_csv("datasets/movies_final.csv")

In [ ]:
import ast

In [49]:
finale = "directors.csv"

fields = ["gender", "name"]
dict_directors = {}
workedon = {}

def get_directors(row):
    if not pd.notna(row["crew"]) or type(row["crew"]) != str:
        return
    
    listi = ast.literal_eval(row["crew"])
    
    if listi is None:
        return
    
    global workedon
    
    for i in listi:
        if i['job'].lower() != "director":
            continue
        
        defdic = {}
        if i["id"] in list(workedon.keys()):
            workedon[i["id"]] += [row["id"]]
        else:
            for k in fields:
                if k in i:
                    defdic[k] = i[k]
            workedon[i["id"]] = [row["id"],]
            dict_directors[i["id"]] = defdic


In [50]:
crewmates.apply(get_directors, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
45471    None
45472    None
45473    None
45474    None
45475    None
Length: 45476, dtype: object

In [51]:
dict_directors

{7879: {'gender': 2, 'name': 'John Lasseter'},
 4945: {'gender': 2, 'name': 'Joe Johnston'},
 26502: {'gender': 2, 'name': 'Howard Deutch'},
 2178: {'gender': 2, 'name': 'Forest Whitaker'},
 56106: {'gender': 2, 'name': 'Charles Shyer'},
 638: {'gender': 2, 'name': 'Michael Mann'},
 2226: {'gender': 2, 'name': 'Sydney Pollack'},
 18357: {'gender': 2, 'name': 'Peter Hewitt'},
 37710: {'gender': 2, 'name': 'Peter Hyams'},
 10702: {'gender': 2, 'name': 'Martin Campbell'},
 3026: {'gender': 2, 'name': 'Rob Reiner'},
 14639: {'gender': 2, 'name': 'Mel Brooks'},
 21879: {'gender': 2, 'name': 'Simon Wells'},
 1152: {'gender': 2, 'name': 'Oliver Stone'},
 16938: {'gender': 2, 'name': 'Renny Harlin'},
 1032: {'gender': 2, 'name': 'Martin Scorsese'},
 1614: {'gender': 2, 'name': 'Ang Lee'},
 3110: {'gender': 1, 'name': 'Allison Anders'},
 3111: {'gender': 2, 'name': 'Alexandre Rockwell'},
 2294: {'gender': 2, 'name': 'Robert Rodriguez'},
 138: {'gender': 2, 'name': 'Quentin Tarantino'},
 4489: {

In [52]:
df_directors = pd.DataFrame.from_dict(dict_directors,orient="index")
df_directors.head(1)

,gender,name
7879,2,John Lasseter


In [53]:
for k in workedon.keys():
    workedon[k] = str(workedon[k])

In [54]:
df_directors["directions"] = pd.DataFrame.from_dict(workedon, orient="index", dtype=str)

In [55]:
df_directors.to_csv(finale, index_label="id")